In [1]:
from dotenv import load_dotenv
import dspy

from agentic_system.agents import CFEfficacyAgent
from agentic_system.litl_data.litl_utils import load_efficacy_devset

load_dotenv("../.env")
lm = dspy.LM("gemini/gemini-2.5-pro", temperature=0.5, cache=False, max_tokens=25000)
dspy.settings.configure(
    lm=lm,
    track_usage=True,
)

In [2]:
import random

DEVSET_MULTIPLIER = 5
# N_DEVSET_SAMPLES = 10

efficacy_devset = load_efficacy_devset(uniform_efficacy=True)
efficacy_devset = efficacy_devset * DEVSET_MULTIPLIER
# efficacy_devset = random.sample(efficacy_devset, N_DEVSET_SAMPLES)

len(efficacy_devset)

125

In [3]:
from dspy.evaluate import Evaluate
import time

N_THREADS = 10
MAX_ITERS = 10


def efficacy_accuracy(example, pred, trace=None):
    "Calculate accuracy from prediction"
    abs_error = abs(example.cf_efficacy - float(pred.predicted_efficacy))
    accuracy = 1 - abs_error
    return accuracy


evaluator = Evaluate(
    devset=efficacy_devset,
    num_threads=N_THREADS,
    display_progress=True,
    provide_traceback=True,
)

efficacy_agent = CFEfficacyAgent(max_iters=MAX_ITERS)

start_time = time.time()
evaluation = evaluator(efficacy_agent, metric=efficacy_accuracy)
end_time = time.time()

Average Metric: 19.73 / 28 (70.5%):  22%|██▏       | 28/125 [03:16<08:22,  5.18s/it]

2025/09/17 16:43:14 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=25000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.5)  if the reason for truncation is repetition.


Average Metric: 30.26 / 44 (68.8%):  35%|███▌      | 44/125 [05:00<04:36,  3.41s/it]

2025/09/17 16:44:50 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=25000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.5)  if the reason for truncation is repetition.


Average Metric: 82.86 / 125 (66.3%): 100%|██████████| 125/125 [13:36<00:00,  6.54s/it]

2025/09/17 16:53:19 INFO dspy.evaluate.evaluate: Average Metric: 82.86158328706732 / 125 (66.3%)


In [4]:
runtime = end_time - start_time
cost = sum([x["cost"] for x in lm.history])

errors = []
confidences = []
step_counts = []
for result in evaluation.results:
    errors.append(abs(result[0].cf_efficacy - result[1].predicted_efficacy))
    confidences.append(result[1].confidence)
    step_counts.append(len(result[1].trajectory) // 4)

print(f"Stats from {len(evaluation.results)} runs:")
print("------------------------")
print("Total Runtime (s):", runtime)
print("Total Cost (USD):", cost)
print("Mean Absolute Error:", sum(errors) / len(errors))
print("Mean Confidence:", sum(confidences) / len(confidences))
print("Mean Step Count:", sum(step_counts) / len(step_counts))

Stats from 125 runs:
------------------------
Total Runtime (s): 816.9684901237488
Total Cost (USD): 18.520130937500003
Mean Absolute Error: 0.3371073337034615
Mean Confidence: 0.8228
Mean Step Count: 5.2
